# Quickmodel
A simple command project I'm putting together. Goal: User runs the file with a specific search term, and the program returns a pickled ML model that is trained to recognize images of the search term. 

In [16]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import matplotlib.pyplot as plt
import os
img_dir = os.listdir('../data/')
img_count = len(img_dir)
type(img_dir)

list

First, let's generate our dataset from our existing files. Luckily, ``keras`` has a nice way of implementing this from their Image library.

In [28]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.3)
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE=20
STEPS_PER_EPOCH = np.ceil(img_count/BATCH_SIZE)

train_generator = image_generator.flow_from_directory(directory='../data',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = ['Eggs', 'NOT-Eggs'], #should come from script parameter
                                                     subset='training',
                                                     class_mode='binary') 
validation_generator = image_generator.flow_from_directory(directory='../data',
                                                          target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                          classes= ['Eggs', 'NOT-Eggs'],
                                                          subset='validation',
                                                          class_mode='binary')

Found 117 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


Now that we have our dataset, we need to construct a model and train it.

In [26]:
model = keras.models.Sequential([
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


Now, let's train and test our model. Keep in mind, the dataset is quite small so accuracy might be low. We will probably need to generate more augmented data. 

In [29]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = 3)


Epoch 1/3
5/5 [==============================] - 12s 2s/step - loss: 6.3779 - accuracy: 0.5906 - val_loss: 6.4687 - val_accuracy: 0.6000
Epoch 2/3
5/5 [==============================] - 13s 3s/step - loss: 5.9346 - accuracy: 0.6242 - val_loss: 5.9097 - val_accuracy: 0.6000
Epoch 3/3
5/5 [==============================] - 12s 2s/step - loss: 6.5426 - accuracy: 0.5797 - val_loss: 5.5370 - val_accuracy: 0.6000
